In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns


In [4]:
df = pd.read_csv('flights.csv')
df.info()

/tmp/ipykernel_51574/493874064.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('flights.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [ ]:
def prep_data(df):
    delay = []
    for row in df['ARRIVAL_DELAY']:
        if row > 60:
            delay.append(3)
        elif row > 30:
            delay.append(2)
        elif row > 15:
            delay.append(1)
        else:
            delay.append(0)
    df['delay'] = delay
    df=df.drop(['YEAR','FLIGHT_NUMBER','AIRLINE','DISTANCE','TAIL_NUMBER','TAXI_OUT','SCHEDULED_TIME','DEPARTURE_TIME','WHEELS_OFF','ELAPSED_TIME','AIR_TIME','WHEELS_ON','DAY_OF_WEEK','TAXI_IN','CANCELLATION_REASON','ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', "CANCELLED"], axis=1)
    df.dropna(inplace = True)
    
    return df
    

In [11]:
df.value_counts('delay')

delay
1    390801
3    319092
2    313605
0     39941
Name: count, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
data = df.values
X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
scaled_features = StandardScaler().fit_transform(X_train, X_test)
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)


In [18]:
from sklearn import metrics
import numpy as np

# Predictions
y_pred = clf.predict(X_test)

# Mean Absolute Error (MAE)
print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred), 3))

# Mean Squared Error (MSE)
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred), 3))

# Root Mean Squared Error (RMSE)
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)), 3))

# R2 Score
print('R2_score:', round(metrics.r2_score(y_test, y_pred), 6))

# Root Mean Squared Log Error (RMSLE)
print('Root Mean Squared Log Error (RMSLE):', round(np.sqrt(metrics.mean_squared_log_error(y_test, y_pred)), 3))


pred_prob = clf.predict_proba(X_test)
print('AUC Score:', round(roc_auc_score(y_test, pred_prob, multi_class='ovr'), 3))

Mean Absolute Error (MAE): 0.004
Mean Squared Error (MSE): 0.004
Root Mean Squared Error (RMSE): 0.066
R2_score: 0.994512
Root Mean Squared Log Error (RMSLE): 0.027
AUC Score: 0.997
